#Defining directories storing images

In [1]:
!mkdir diff-run

mkdir: cannot create directory ‘diff-run’: File exists


In [2]:
!mkdir diff-run/images

mkdir: cannot create directory ‘diff-run/images’: File exists


# Importing modules

In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [4]:
torch.manual_seed(1)

In [5]:
writer = SummaryWriter('diff-run/py-gan')

# Setting up device

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128

# Loading dataset and preprocessing

In [7]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])])
train_dataset = datasets.FashionMNIST(root='./data/', train=True, transform=train_transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [8]:
image_shape = (1, 28, 28)
image_dim = int(np.prod(image_shape))
latent_dim = 100

# Building the Generator model

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(nn.Linear(latent_dim, 128),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(128, 256),
                      nn.BatchNorm1d(256, 0.8),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(256, 512),
                      nn.BatchNorm1d(512, 0.8),
                                    nn.LeakyReLU(0.2, inplace=True),
                      nn.Linear(512, 1024),
                      nn.BatchNorm1d(1024, 0.8),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(1024, image_dim),
                                    nn.Tanh())

    def forward(self, noise_vector):
        image = self.model(noise_vector)
        image = image.view(image.size(0), *image_shape)
        return image

# Building the Discriminator model

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(nn.Linear(image_dim, 512),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(512, 256),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(256, 1),
                                    nn.Sigmoid())

    def forward(self, image):
        image_flattened = image.view(image.size(0), -1)
        result = self.model(image_flattened)
        return result

In [11]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [12]:
summary(generator, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]          12,928
         LeakyReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 256]          33,024
       BatchNorm1d-4                  [-1, 256]             512
         LeakyReLU-5                  [-1, 256]               0
            Linear-6                  [-1, 512]         131,584
       BatchNorm1d-7                  [-1, 512]           1,024
         LeakyReLU-8                  [-1, 512]               0
            Linear-9                 [-1, 1024]         525,312
      BatchNorm1d-10                 [-1, 1024]           2,048
        LeakyReLU-11                 [-1, 1024]               0
           Linear-12                  [-1, 784]         803,600
             Tanh-13                  [-1, 784]               0
Total params: 1,510,032
Trainable param

In [13]:
summary(discriminator, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 512]         401,920
         LeakyReLU-2                  [-1, 512]               0
            Linear-3                  [-1, 256]         131,328
         LeakyReLU-4                  [-1, 256]               0
            Linear-5                    [-1, 1]             257
           Sigmoid-6                    [-1, 1]               0
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.04
Estimated Total Size (MB): 2.05
----------------------------------------------------------------


# Defining loss function (here binary cross entropy error)

In [14]:
adversarial_loss = nn.BCELoss()

# Optimization of Generator and Discriminator

In [15]:
learning_rate = 0.0002
G_optimizer = optim.Adam(generator.parameters(), lr = learning_rate, betas=(0.5, 0.999))
D_optimizer = optim.Adam(discriminator.parameters(), lr = learning_rate, betas=(0.5, 0.999))

In [16]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

#Training the GAN

In [17]:
num_epochs = 75
D_loss_plot, G_loss_plot = [], []
for epoch in range(1, num_epochs+1):

    D_loss_list, G_loss_list = [], []

    for index, (real_images, _) in enumerate(train_loader):
        D_optimizer.zero_grad()     #zero out the old gradients
        real_images = real_images.to(device)
        real_target = Variable(torch.ones(real_images.size(0), 1).to(device))
        fake_target = Variable(torch.zeros(real_images.size(0), 1).to(device))

        # Training Discriminator on Real Data
        D_real_loss = adversarial_loss(discriminator(real_images), real_target)

        # noise vector sampled from a normal distribution
        noise_vector = Variable(torch.randn(real_images.size(0), latent_dim).to(device))
        noise_vector = noise_vector.to(device)
        generated_image = generator(noise_vector)

        # Training Discriminator on Fake Data
        D_fake_loss = adversarial_loss(discriminator(generated_image),\
                                     fake_target)

        D_total_loss = D_real_loss + D_fake_loss
        D_loss_list.append(D_total_loss)
        D_total_loss.backward()
        D_optimizer.step()

        # Train Generator on Discriminator's output
        G_optimizer.zero_grad()     #zero out the old gradients
        generated_image = generator(noise_vector)
        G_loss = adversarial_loss(discriminator(generated_image), real_target)
        G_loss_list.append(G_loss)

        G_loss.backward()
        G_optimizer.step()
        d = generated_image.data

        writer.add_scalar('Discriminator Loss',
                            D_total_loss,
                            epoch * len(train_loader) + index)

        writer.add_scalar('Generator Loss',
                            G_loss,
                            epoch * len(train_loader) + index)


    print('Epoch: [%d/%d]: D_loss: %.3f, G_loss: %.3f' % (
            (epoch), num_epochs, torch.mean(torch.FloatTensor(D_loss_list)),\
             torch.mean(torch.FloatTensor(G_loss_list))))

    D_loss_plot.append(torch.mean(torch.FloatTensor(D_loss_list)))
    G_loss_plot.append(torch.mean(torch.FloatTensor(G_loss_list)))
    save_image(generated_image.data[:90], 'diff-run/images/Final_Sample_%d'%epoch + '.png', nrow=10, normalize=True)

Epoch: [1/75]: D_loss: 1.019, G_loss: 1.369
Epoch: [2/75]: D_loss: 0.886, G_loss: 2.059
Epoch: [3/75]: D_loss: 0.883, G_loss: 1.989
Epoch: [4/75]: D_loss: 1.021, G_loss: 1.658
Epoch: [5/75]: D_loss: 1.100, G_loss: 1.415
Epoch: [6/75]: D_loss: 1.130, G_loss: 1.396
Epoch: [7/75]: D_loss: 1.126, G_loss: 1.347
Epoch: [8/75]: D_loss: 1.150, G_loss: 1.279
Epoch: [9/75]: D_loss: 1.158, G_loss: 1.279
Epoch: [10/75]: D_loss: 1.199, G_loss: 1.173
Epoch: [11/75]: D_loss: 1.210, G_loss: 1.142
Epoch: [12/75]: D_loss: 1.231, G_loss: 1.113
Epoch: [13/75]: D_loss: 1.229, G_loss: 1.087
Epoch: [14/75]: D_loss: 1.226, G_loss: 1.110
Epoch: [15/75]: D_loss: 1.234, G_loss: 1.092
Epoch: [16/75]: D_loss: 1.248, G_loss: 1.071
Epoch: [17/75]: D_loss: 1.243, G_loss: 1.045
Epoch: [18/75]: D_loss: 1.254, G_loss: 1.038
Epoch: [19/75]: D_loss: 1.247, G_loss: 1.028
Epoch: [20/75]: D_loss: 1.244, G_loss: 1.041
Epoch: [21/75]: D_loss: 1.247, G_loss: 1.026
Epoch: [22/75]: D_loss: 1.242, G_loss: 1.027
Epoch: [23/75]: D_l

# Making directory to store GAN generated images

In [18]:
!mkdir diff-run/GANimages

#Saving the GAN model

In [19]:
# Load the trained GAN model
torch.save(generator.state_dict(), 'generator_model.pth')
generator.load_state_dict(torch.load('generator_model.pth'))

<All keys matched successfully>

# Creating samples from the trained GAN model

In [20]:
num_samples = 128
latent_dim = 100
random_input = torch.randn(num_samples, latent_dim).to(device)

# Generate samples from the generator
generator.eval()  # Set the generator to evaluation mode
generated_samples = generator(random_input)

# Specify the custom save location
save_location = 'diff-run/GANimages/Final_generated_samples.png'

# Save the generated samples
save_image(generated_samples, save_location, normalize=True)